In [ ]:
# 3. Build Model (Dual-Input ConvLSTM)

def build_model(input_shape_x, input_shape_t, output_shape):
    # Input 1: History (Batch, 30, 64, 2, 1)
    input_x = layers.Input(shape=input_shape_x, name='history_input')
    
    # Input 2: Terminal Schedule (Batch, 15, 2, 1)
    input_t = layers.Input(shape=input_shape_t, name='terminal_input')
    
    # --- Encoder (ConvLSTM Branch) ---
    # Layer 1
    x = layers.ConvLSTM2D(
        filters=32,
        kernel_size=(3, 1),
        padding='same',
        return_sequences=True,
        activation='relu'
    )(input_x)
    
    # Layer 2
    x = layers.ConvLSTM2D(
        filters=32,
        kernel_size=(3, 1),
        padding='same',
        return_sequences=False, # Compress time dimension
        activation='relu'
    )(x)
    
    # Flatten Spatial Features
    x = layers.Flatten()(x)
    
    # --- Fusion ---
    # Flatten Schedule
    t = layers.Flatten()(input_t)
    
    # Concatenate
    combined = layers.Concatenate()([x, t])
    
    # --- Decoder (Projector) ---
    # Calculate output dimensions
    # Output shape: (15, 64, 2, 1)
    out_steps = output_shape[0]
    out_stations = output_shape[1]
    out_dirs = output_shape[2]
    out_channels = output_shape[3]
    
    flat_output_size = out_steps * out_stations * out_dirs * out_channels
    
    z = layers.Dense(flat_output_size, activation='sigmoid')(combined) # Sigmoid for [0, 1] output
    
    # Reshape to target shape
    output = layers.Reshape(output_shape)(z)
    
    model = keras.Model(inputs=[input_x, input_t], outputs=output)
    return model

# Define shapes
# X: (30, 64, 2, 1)
input_shape_x = (LOOKBACK_MINS, data.shape[1], data.shape[2], data.shape[3])
# T: (15, 2, 1)
input_shape_t = (FORECAST_MINS, schedule.shape[1], schedule.shape[2])
# Y: (15, 64, 2, 1)
output_shape = (FORECAST_MINS, data.shape[1], data.shape[2], data.shape[3])

model = build_model(input_shape_x, input_shape_t, output_shape)
model.summary()

# Compile
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='mse',
    metrics=[keras.metrics.RootMeanSquaredError()]
)


In [ ]:
# 4. Train

callbacks = [
    keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True)
]

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=callbacks
)

# Plot History
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss (MSE)')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['root_mean_squared_error'], label='Train RMSE')
plt.plot(history.history['val_root_mean_squared_error'], label='Val RMSE')
plt.title('RMSE')
plt.legend()
plt.show()
